In [ ]:
# Standard Imports
import pandas as pd
import matplotlib as plt
import seaborn as sns

# Add additional imports under here
import numpy as np
from pathlib import Path
import re
import stopwordsiso as stopwords
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googletrans import Translator
import spacy

c:\Users\116su\source\repos\Assignment_3_Code\venv\Lib\site-packages\stopwordsiso\_core.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [9]:
#loading
df = pd.read_excel("lexicon_6000 words.xlsx")

#keep a copy of the original columns
original_Cols = df.columns.tolist()
print(original_Cols)

df.head()

['CILUBA', 'French', 'Score', 'Sentiment', 'Nature']


,CILUBA,French,Score,Sentiment,Nature
0,Akaja,Arrange,1.0,Positif,Verbe
1,Akajilula,Rearrange,1.0,Positif,Verbe
2,Akula,Parle,2.0,Positif,Verbe
3,Akulula,Reparle,2.0,Positif,Verbe
4,Aluja,Remet,3.0,Positif,Verbe


In [10]:
dup_Count = df.duplicated().sum()
print(dup_Count)

if dup_Count > 0:
    print("dataset size:", len(df)) 
    df = df.drop_duplicates()
    print("new dataset size:", len(df))

#taking away any duplication/ spacing errors
col_Strings = df.select_dtypes(include=['object']).columns
for col in col_Strings:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].str.replace(r'\s+','', regex=True)

317
dataset size: 6963
new dataset size: 6646


In [11]:
new_Languages = ["English", "Zulu", "Afrikaans", "Sepedi", "Xhosa", "Shona"]

for lang in new_Languages:
    if lang not in df.columns:
        df[lang] = np.nan

df.head()

,CILUBA,French,Score,Sentiment,Nature,English,Zulu,Afrikaans,Sepedi,Xhosa,Shona
0,Akaja,Arrange,1.0,Positif,Verbe,NaN,NaN,NaN,NaN,NaN,NaN
1,Akajilula,Rearrange,1.0,Positif,Verbe,NaN,NaN,NaN,NaN,NaN,NaN
2,Akula,Parle,2.0,Positif,Verbe,NaN,NaN,NaN,NaN,NaN,NaN
3,Akulula,Reparle,2.0,Positif,Verbe,NaN,NaN,NaN,NaN,NaN,NaN
4,Aluja,Remet,3.0,Positif,Verbe,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from deep_translator import GoogleTranslator

def translate_French_To_English(text):
    if pd.isna(text) or text.strip=="":
        return np.man
    try:
        return GoogleTranslator(source='fr', target='en').translate(text)
    except Exception:
        return np.nan

def translate_English_To_Language(text, language_Code):
    if pd.isna(text) or text.strip()=="":
        return np.man
    try:
        return GoogleTranslator(source="en", target=language_Code).translate(text)
    except Exception:
        return np.nan

In [ ]:
from tqdm import tqdm

tqdm.pandas()

df["English"] = df["French"].progress_apply(translate_French_To_English)

In [ ]:
print("Errors occured:", df["English"].isna().sum())
print("Unique English words:", df["English"].nunique(dropna=True))

Errors occured: 0
Unique English words: 3864


In [ ]:
language_Code = {"Zulu":"zu", "Afrikaans":"af", "Sepedi":"nso", "Xhosa":"xh", "Shona":"sn"}

for lang, code in language_Code.items():
    df[lang] = df["English"].progress_apply(lambda x: translate_English_To_Language(x, code))

100%|████████████████████████████████████████████████████████████████████████████| 6632/6632 [2:19:04<00:00,  1.26s/it]


In [ ]:
df.head(20)

,CILUBA,French,Score,Sentiment,Nature,English,Zulu,Afrikaans,Sepedi,Xhosa,Shona
0,Akaja,Arrange,1.0,Positif,Verbe,Arrange,Hlela,Reël,Beakanya,Cwangcisa,Ronga
1,Akajilula,Rearrange,1.0,Positif,Verbe,Rear range,Ibanga langemuva,Grootafstand,Range ya ka morao .,Uluhlu lwangasemva,Kumashure
2,Akula,Parle,2.0,Positif,Verbe,Speak,Khuluma,Praat,Bolela,Thetha,Taura
3,Akulula,Reparle,2.0,Positif,Verbe,Speak again,Khuluma futhi,Praat weer,Bolela gape .,Thetha kwakhona,Taura zvakare
4,Aluja,Remet,3.0,Positif,Verbe,Hands over,Izandla ngaphezulu,Hande om,diatla godimo ga .,Izandla ngaphezulu,Maoko pamusoro
5,Amba,Dis,3.0,Positif,Verbe,Say,Khuluma,Sê,Bolela,Yithi,Taura
6,Ambakaja,Supperpose,3.0,Positif,Verbe,Suppose,Cabanga,Veronderstel,Nagana,Cinga,Ngatitii
7,Ambula,Ramasse,4.0,Positif,Verbe,Pick up,Phakamisa,Optel,Topa,Phakamisa,Nhonga
8,Ambuluja,Depeche,4.0,Positif,Verbe,Depeche,I-depeche,Depeche,Depeche .,I-dope,Zvinorevache
9,Ambulula,Repete,9.0,Positif,Verbe,Repeated,-Phindaphiwe,Herhaal,pheta-pheta .,Iphindaphindwe,Kudzokororwa


In [ ]:
#missing translations (NaN)
for lang in ["Zulu","Afrikaans","Sepedi","Xhosa","Shona"]:
    print(f"{lang}: missing {df[lang].isna().sum()}")

# unique counter
df[["English","Zulu","Afrikaans","Sepedi","Xhosa","Shona"]].nunique()

Zulu: missing 1
Afrikaans: missing 0
Sepedi: missing 0
Xhosa: missing 1
Shona: missing 1


English      3864
Zulu         3513
Afrikaans    3647
Sepedi       3376
Xhosa        3367
Shona        3156
dtype: int64

In [ ]:
df_lexicon = pd.read_excel('./lexicon_dataset/expanded_lexicon.xlsx')

sentiment_map = {
    # French / variants meaning Positive
    "Positive": "positive",
    "Positf": "positive",
    "Positidf": "positive",
    "Positiveouneutre": "positive",
    "Positiveetnaturelle": "positive",
    "Trespositive": "positive",
    "Trèspositive": "positive",

    # French / variants meaning Negative
    "Negative": "negative",
    "Négative": "negative",
    "Tresnegative": "negative",
    "Trèsnegative": "negative",

    # Neutral / misspelled variants
    "Neutre": "neutral",
    "Neutrre": "neutral",
    "Neuitre": "neutral",

    # Ambiguous terms
    "Ambivalent": "neutral",   # interpret ambivalent as neutral
}

df_lexicon['Sentiment'] = df_lexicon['Sentiment'].map(sentiment_map)
df_lexicon.to_excel('./lexicon_dataset/expanded_lexicon_cleaned.xlsx', index=False)
print(df_lexicon["Sentiment"].value_counts())

Sentiment
positive    4361
neutral     1121
negative    1116
Name: count, dtype: int64


In [ ]:
def tokenize_text(text):
    text = text.translate(str.maketrans('', '', '()[]{}1234567890.?!')).lower().split('\t')
    return text

if stopwords.has_lang("zu"):
    zulu_stopwords = stopwords.stopwords("zu") # Used to filter out common words
else:
    print("No stopwords found for Zulu")

df_lexicon_clean = pd.read_excel('./lexicon_dataset/expanded_lexicon_cleaned.xlsx')
print('Pre cleaning dataset length: ', len(df_lexicon_clean))
columns = []
for col in df_lexicon_clean.columns:
    if col != 'Score':
        columns.append(col)

for col in df_lexicon_clean.columns:
    if df_lexicon_clean[col].dtype == 'object':
        df_lexicon_clean[col] = df_lexicon_clean[col].str.lower()
        
df_lexicon_clean[columns] = df_lexicon_clean[columns].replace(np.nan, None)
           
lang_key = {
    "English": "en",
    "Zulu": "zu",
    "Afrikaans": "af",
    "Sepedi": "nso",
    "Xhosa": "xh",
    "Shona": "sn"
}
translator = Translator()
def retranslator(index):
    for col in ["English","Zulu","Afrikaans","Sepedi","Xhosa","Shona"]:
        if df_lexicon_clean.at[index, col] is None:
            detect = translator.detect(df_lexicon_clean.at[index, 'French'])
            if detect.lang != 'fr':
                try:
                    translated = translator.translate(df_lexicon_clean.at[index, 'French'], src=detect.lang, dest=lang_key[col])
                    df_lexicon_clean.at[index, col] = translated.text
                    print(f'translated {df_lexicon_clean.at[index, 'French']} to {translated.text}')
                except:
                    print(f'could not translate {df_lexicon_clean.at[index, 'French']} to {col}')
                    continue

for index, row in df_lexicon_clean.iterrows():
    retranslator(index)

for index, row in df_lexicon_clean.iterrows():
    if row['Zulu'] in zulu_stopwords:
        df_lexicon_clean = df_lexicon_clean.drop(index)

newSentiments = []
for index, row in df_lexicon_clean.iterrows():   
    if row['Score'] > 0.5:
        newSentiments.append('postive')
    elif row['Score'] < -0.5:
        newSentiments.append('negative')
    else:
        newSentiments.append('neutral')

df_lexicon_clean['Sentiment'] = newSentiments
df_lexicon_clean = df_lexicon_clean.drop_duplicates()
print('Post cleaning dataset length: ', len(df_lexicon_clean))
df_lexicon_clean.to_excel('./lexicon_dataset/expanded_lexicon_cleaned.xlsx', )

Pre cleaning dataset length:  6632
translated populace to bevolking
Post cleaning dataset length:  6451


In [111]:
# Extracting different text features from the corpus extracted from SADII of news articles in isiZulu
file = open('SADII-Ext.Caps.POS.2024-01-31.zu.txt', 'r', encoding='utf-8')
lines = file.read()
lines = lines.splitlines()
file.close()
    
df_corpus = pd.DataFrame(lines, columns=['isiZulu'])
df_corpus['isiZulu'] = df_corpus['isiZulu'].str.strip()

print("Total text samples from corpus: ", len(df_corpus))
print(df_corpus.head())

Total text samples from corpus:  4640
            isiZulu
0          <LINE 1>
1  Abacwaningi\tN02
2          bathi\tV
3      ukujezisa\tV
4   ngokushaya\tADV


In [112]:
# Cleaning and Tokenization
df_corpus_text = pd.DataFrame(columns=['isiZulu'])

# Reverse engineering the text samples based on <line> tags
text = ''
for corpus_text in df_corpus['isiZulu']:
    if '<line' in corpus_text.lower():
        if text:
            text = text.strip().replace('"', '')
            df_corpus_text.loc[len(df_corpus_text)] = text
            
        text = ''
        continue
    
    tokens = tokenize_text(corpus_text)
    text += tokens[0] + ' '

print("Total text samples: ", len(df_corpus_text))
print(df_corpus_text.head())

Total text samples:  465
                                             isiZulu
0  abacwaningi bathi ukujezisa ngokushaya noma ng...
1  abafundi abasebancane kakhulu bona bangakuthok...
2                                       abangaphoqwa
3  abantu abaningi bayaluthokozela lolu hlelo kwa...
4  abantu abasha bakhala ngemfundo yamahhala kany...


In [115]:
english_text = []
sentiment_result = []

print('Corpus translation in progress')
for sentence in df_corpus_text['isiZulu']:
    translated_text = GoogleTranslator(source='auto', target='en').translate(sentence)
    analyzer = SentimentIntensityAnalyzer()
    sentiment_dict = analyzer.polarity_scores(translated_text)
    
    english_text.append(translated_text)
    if sentiment_dict['compound'] >= 0.05:
        sentiment_result.append('Positive')     
    elif sentiment_dict['compound'] <= - 0.05:
        sentiment_result.append('Negative')   
    else:
        sentiment_result.append('Neutral')   

df_corpus_text['English'] = english_text
df_corpus_text['Sentiment'] = sentiment_result
print(df_corpus_text.head())

Corpus translation in progress
                                             isiZulu  \
0  abacwaningi bathi ukujezisa ngokushaya noma ng...   
1  abafundi abasebancane kakhulu bona bangakuthok...   
2                                       abangaphoqwa   
3  abantu abaningi bayaluthokozela lolu hlelo kwa...   
4  abantu abasha bakhala ngemfundo yamahhala kany...   

                                             English Sentiment  
0  researchers say that punishing by beating, no ...  Negative  
1  Younger readers can enjoy reading texts with l...  Positive  
2                               who cannot be forced  Positive  
3  Many people enjoy this program because even a ...  Positive  
4  young people are complaining about free educat...  Negative  


In [116]:
print("Number of pre-cleansed samples: ", len(df_corpus_text))
# Clean untranslated text samples
for index, row in df_corpus_text.iterrows():
    detect = translator.detect(row['English'])
    if detect.lang != 'en':
        try:
            translated = translator.translate(row['English'], src=detect.lang, dest='en')
            df_corpus_text.at[index, 'English'] = translated.text
        except:
            df_corpus_text.drop(index, inplace=True)

print("Number of post-cleansed samples: ", len(df_corpus_text))
df_corpus_text.to_csv('./corpus_datasets/Zulu_to_English_Corpus.csv', index=False)

Number of pre-cleansed samples:  465
Number of post-cleansed samples:  465


In [ ]:
from collections import Counter
import math


df_lexicon = pd.read_csv('.\corpus_datasets\Zulu_to_English_Corpus.csv')
#need to expand stop words list
zulu_stopwords = set([
    'nokuthi', 'yokuthi', 'kanti', 'futhi', 'ke', 'na', 'kakhulu',
    'uma', 'ku', 'ngokuthi', 'lokhu', 'konke', 'zonke'
]) 


df_words = []
for idx, row in df_lexicon.iterrows():
    words = row['isiZulu'].lower().split()
    for w in words:
        # Remove punctuation
        w = w.strip('.,!?;"()[]{}')
        # Skip empty strings and stopwords
        if w and w not in zulu_stopwords:
            df_words.append({'tokens': w, 'label': row['Sentiment'].lower()})

df_tokens_labeled = pd.DataFrame(df_words)

print(f"Total tokens for PMI (after removing stopwords): {len(df_tokens_labeled)}")

pos_tokens = df_tokens_labeled[df_tokens_labeled['label'] == 'positive']['tokens'].tolist()
neg_tokens = df_tokens_labeled[df_tokens_labeled['label'] == 'negative']['tokens'].tolist()

pos_counts = Counter(pos_tokens)
neg_counts = Counter(neg_tokens)
all_counts = Counter(df_tokens_labeled['tokens'])

total_pos = sum(pos_counts.values())
total_neg = sum(neg_counts.values())
total_all = sum(all_counts.values())


pmi_data = []
for word in all_counts:
    p_word = all_counts[word] / total_all
    pmi_pos = math.log2((pos_counts[word]/total_pos)/p_word) if word in pos_counts else 0
    pmi_neg = math.log2((neg_counts[word]/total_neg)/p_word) if word in neg_counts else 0
    pmi_diff = pmi_pos - pmi_neg
    pmi_data.append((word, pmi_pos, pmi_neg, pmi_diff))

df_pmi = pd.DataFrame(pmi_data, columns=['word', 'PMI_Pos', 'PMI_Neg', 'PMI_Diff'])

positive_candidates = df_pmi[df_pmi['PMI_Diff'] > 1].sort_values(by='PMI_Diff', ascending=False)
negative_candidates = df_pmi[df_pmi['PMI_Diff'] < -1].sort_values(by='PMI_Diff', ascending=True)

print("Top positive candidate words:")
print(positive_candidates.head(20))

print("\nTop negative candidate words:")
print(negative_candidates.head(20))

Total tokens for PMI (after removing stopwords): 5686
Top positive candidate words:
              word   PMI_Pos   PMI_Neg  PMI_Diff
2000         wakhe  0.585026 -1.425497  2.010523
80     zokuxhumana  0.585026 -1.425497  2.010523
1195       iqiniso  0.906954 -0.840535  1.747489
778         ithuba  0.906954 -0.840535  1.747489
384         ngakho  0.684562 -1.062927  1.747489
132       imidlalo  0.684562 -1.062927  1.747489
79    ezinkundleni  0.684562 -1.062927  1.747489
904          ukuze  0.453781 -1.194172  1.647953
509         impilo  0.585026 -0.840535  1.425561
585         sifuna  0.585026 -0.840535  1.425561
505         phansi  0.585026 -0.840535  1.425561
426           nabo  0.169988 -1.255572  1.425561
683          isimo  0.848060 -0.577501  1.425561
237           kulo  0.362634 -1.062927  1.425561
774        uthando  0.848060 -0.577501  1.425561
444         ezinye  0.848060 -0.577501  1.425561
447          ngami  0.848060 -0.577501  1.425561
161          ngayo  0.381493 -0.91

<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\vanma\AppData\Local\Temp\ipykernel_5760\3890579250.py:6: SyntaxWarning: invalid escape sequence '\c'
  df_lexicon = pd.read_csv('.\corpus_datasets\Zulu_to_English_Corpus.csv')
